<a href="https://colab.research.google.com/github/veerajalluri/agenticAI/blob/main/hackathon_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# [Step 1] Enable GPU in Colab
# Go to Runtime -> Change runtime type -> Select T4 GPU

# [Step 2] Install dependencies
!pip install -qU transformers accelerate gradio bitsandbytes huggingface_hub

# [Step 3] Authenticate with Hugging Face
from google.colab import userdata
from huggingface_hub import login

# Get your token from https://huggingface.co/settings/tokens
hf_token = userdata.get('HF_TOKEN')  # Add via Colab secrets (left sidebar 🔑)
login(token=hf_token)

# [Step 4] Configure model with proper quantization
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"

# 4-bit quantization config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=hf_token
)

# [Step 5] Create code review function
def code_review_agent(code: str) -> str:
    """
    Analyzes code using DeepSeek model for:
    - Potential bugs
    - Security vulnerabilities
    - Code quality issues
    - Style improvements
    Returns formatted review with line numbers
    """
    system_prompt = """You are a senior software engineer performing code review.
    Analyze the code and provide detailed feedback with:
    1. Critical issues (bugs/security) marked with 🚨
    2. Improvements marked with 💡
    3. Always include line numbers
    4. Suggest concrete fixes
    Format response in Markdown."""

    user_prompt = f"Review this Python code:\npython\n{code}\n"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        temperature=0.3,
        top_p=0.95,
        do_sample=True
    )

    return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

# [Step 6] Test the agent
test_code = """
def calculate_average(nums):
    total = sum(nums)
    return total / len(nums)

print(calculate_average([]))
"""

print("Testing code review agent...\n")
print(code_review_agent(test_code))

# [Step 7] Create Gradio UI
import gradio as gr

def gradio_interface(code):
    review = code_review_agent(code)
    return f"*Code Review Report*\n\n{review}"

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# AI Code Review Agent 🤖")
    with gr.Row():
        code_input = gr.Code(
            label="Input Code",
            language="python",
            lines=10,
            interactive=True
        )
        review_output = gr.Markdown(label="Review Results")

    submit_btn = gr.Button("Analyze Code", variant="primary")
    examples = gr.Examples(
        examples=[test_code],
        inputs=code_input
    )

    submit_btn.click(
        fn=gradio_interface,
        inputs=code_input,
        outputs=review_output
    )

demo.launch(share=True)  # Get public link from output

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Testing code review agent...

🚨 The function calculate_average does not handle the case where the input list is empty. This could lead to a ZeroDivisionError.

💡 The function can be simplified by using the built-in Python function `statistics.mean`.

Line 1: calculate_average(nums)
Line 2: total = sum(nums)
Line 3: return total / len(nums)
Line 4: print(calculate_average([]))

Suggested fix:
```python
import statistics

def calculate_average(nums):
    if len(nums) == 0:
        return 0
    return statistics.mean(nums)

print(calculate_average([]))
```

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0fa1696ddbdd1a52b0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
